In [ ]:
import os, sys
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import colormaps
from scipy.signal import medfilt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import numpy as np
import pandas as pd
import netCDF4 as nc
plt.rcParams.update({
    "text.usetex": True,                # Enable LaTeX text rendering
    "font.family": "serif",             # Use a serif font
    "font.serif": ["Computer Modern"],  # Set font to Computer Modern (LaTeX default)
})
sys.path.append('../src/')
from Biologging_Toolkit.applications.Jerk import Jerk
from Biologging_Toolkit.utils.plot_utils import subplots_centered
import warnings
warnings.filterwarnings("ignore")

In [ ]:
depid = 'ml19_292a'
ds_path = 'D:/individus_brut/individus'
annotation_path = 'D:/drift_dives_SES/'

In [ ]:
ds = nc.Dataset(os.path.join(ds_path, depid, depid+'_sens.nc'))
dive = pd.read_csv(os.path.join(ds_path, depid, depid+'_dive.csv'))
annot = pd.read_csv(os.path.join(annotation_path, depid, 'timestamps.csv'))

In [ ]:
dives = ds['dives'][:].data
jerks = ds['jerk'][:].data
total_jerk = [np.sum(jerks[dives == dive]) for dive in np.unique(dives)]
dive['total_jerk'] = total_jerk

In [ ]:
def successive_detections(series):
    changes = series.diff().fillna(0)
    starts = changes == 1 
    ends = changes == -1
    cumulative = series.cumsum() 
    lengths = cumulative[ends].diff().fillna(cumulative[ends]) 
    result = np.zeros_like(series)
    result[ends[ends].index] = lengths.values
    return pd.Series(result, index=series.index)
def sliding_window_sum(time_series, N):
    result = np.zeros_like(time_series)
    for i in range(len(time_series) - N + 1):
        result[i + N - 1] = np.sum(time_series[i:i + N])
    return result

In [ ]:
labels = np.unique(np.concatenate([annot[col].dropna() for col in ['Annotation','Annotation2','Annotation3']]))
annot_pos = np.searchsorted(dive.begin_time, annot.start_drift)-1
annotations = {label : np.zeros(len(dive)) for label in labels}
'''annotations = {'Blue whale':np.zeros(len(dive))
               'Antarctic blue whale':np.zeros(len(dive)),
               'Sperm whale':np.zeros(len(dive)),
               'Sei whale':np.zeros(len(dive)),
               'Humpback whale':np.zeros(len(dive)),
               'Fin whale':np.zeros(len(dive)),
               'Minke whale':np.zeros(len(dive))}'''
print(labels)

In [ ]:
idx = 1
for i, row in annot.iterrows() :
    det = row[['Annotation', 'Annotation2', 'Annotation3']].to_numpy()
    conf = row[['Indice', 'Indice2', 'Indice3']].to_numpy()
    det = det[conf >= idx]
    for _det in det :
        if _det in ['Antarctic blue whale', 'Dcall', 'Blue whale'] :
            annotations['Antarctic blue whale'][annot_pos[i]] = 1
        elif _det=='Minke whale':
            annotations['Minke whale'][annot_pos[i]] = 1
        elif _det=='Spermwhale':
            annotations['Spermwhale'][annot_pos[i]] = 1
        elif _det=='Fin whale':
            annotations['Fin whale'][annot_pos[i]] = 1
        elif _det=='Humpback whale':
            annotations['Humpback whale'][annot_pos[i]] = 1
        elif _det=='Sei whale':
            annotations['Sei whale'][annot_pos[i]] = 1
        elif _det in['Odontocete clicks', 'Odontocete buzz', 'Odontocete whistle'] :
            annotations['Odontocete clicks'][annot_pos[i]] = 1
annotations = pd.DataFrame(annotations)
for key in list(annotations.keys()) :
    dive[key] = successive_detections(annotations[key])

In [ ]:
temp_jerk = sliding_window_sum(dive.total_jerk, 35)
dive['PCA'] = temp_jerk
np.corrcoef(dive.PCA, dive[annotations.columns].sum(axis = 1))

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(12, 6), subplot_kw={'projection': ccrs.PlateCarree()})
ax = ax.flatten()
columns = list(dive[annotations.columns].sum()[dive[annotations.columns].sum() > 0].index)
temp_jerk = sliding_window_sum(dive.total_jerk, 10)
dive['PCA'] = (temp_jerk - np.nanmean(temp_jerk)) / np.nanstd(temp_jerk)
columns.append('PCA')
colors = colormaps.get_cmap('plasma').resampled(len(columns))
for i, elem in enumerate(columns[::-1][-5:-1]) :
    ax[i].set_extent([66, 90, -45, -55], crs=ccrs.PlateCarree())  # Adjust the limits as needed
    ax[i].plot(dive.lon, dive.lat, c = 'k', linewidth = 2, label = elem)
    ax[i].scatter(dive.lon, dive.lat, s = dive[elem] * 50, c = colors(i))
    ax[i].add_feature(cfeature.OCEAN, facecolor='lightblue')
    ax[i].add_feature(cfeature.LAND, edgecolor='black')
    ax[i].add_feature(cfeature.COASTLINE)
    ax[i].add_feature(cfeature.BORDERS, linestyle=':', edgecolor='gray')
    ax[i].gridlines(draw_labels=True)
    ax[i].set_title(f"{elem}")
fig.tight_layout()

In [ ]:
len(columns)

In [ ]:
fig.savefig("C:/Users/grosm/Desktop/thèse/Figures/whale/map_comparison_ml18_294b.pdf")

In [ ]:
fig, ax = plt.subplots(figsize = (8, 8))
dive["begin_time"] = pd.to_datetime(dive["begin_time"], unit="s")
columns = list(dive[annotations.columns].sum()[dive[annotations.columns].sum() > 0].index)
colors = colormaps.get_cmap('viridis').resampled(len(columns))
ax1 = ax.twinx()
ax.scatter(dive.begin_time[dive.total_jerk > 0], dive.total_jerk[dive.total_jerk > 0], 
           alpha = 0.2, c = 'orange', label = 'PCA')
ax.plot(dive.begin_time, medfilt(dive.total_jerk, 101), c = 'orange', label = 'median PCA')
for i, elem in enumerate(columns[::-1]) :
    ax1.plot(dive.begin_time, dive[elem], c = colors(i), linewidth = 3, label = elem)
ax.set_facecolor('whitesmoke')
ax1.grid()
ax.tick_params(axis='x', rotation=45)
handles_ax, labels_ax = ax.get_legend_handles_labels()
handles_ax1, labels_ax1 = ax1.get_legend_handles_labels()
legend = ax.legend(handles_ax + handles_ax1, labels_ax + labels_ax1, loc="upper left")
ax.set_ylabel("Number of prey catch attempts (PCA)")
ax1.set_ylabel("Number of recordings with detection")

In [ ]:
fig.savefig("C:/Users/grosm/Desktop/thèse/Figures/")